In [30]:
"""
Script to render the asset pricing table
"""

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

import warnings
warnings.filterwarnings("ignore")

from environ.constants import (
    DEPENDENT_VARIABLES,
    PROCESSED_DATA_PATH,
    STABLE_DICT,
    TABLE_PATH,
)
from environ.process.asset_pricing.double_sorting import calculate_period_return
from environ.process.asset_pricing.assetpricing_functions import build_dominance_factor, calculate_weekly_returns

def reg_fama_macbeth(data_fama_macbeth, formula:str = "dollar_ret ~ DOM"
) -> pd.DataFrame:
  risk_premia = (data_fama_macbeth
    .groupby(["Year", "Week"])
    .apply(lambda x: smf.ols(
        formula= formula, 
        data=x
      ).fit()
      .params
    )
    .reset_index()
  )

  price_of_risk = (risk_premia
    .melt(id_vars=["Year", "Week"], var_name="factor", value_name="estimate")
    .groupby("factor")["estimate"]
    .apply(lambda x: pd.Series({
        "risk_premium": x.mean(),
        "t_stat": x.mean()/x.std()*np.sqrt(len(x))
      })
    )
    .reset_index()
    .pivot(index="factor", columns="level_1", values="estimate")
    .reset_index()
  )

  price_of_risk_NW = (risk_premia
    .melt(id_vars=["Year", "Week"], var_name="factor", value_name="estimate")
    .groupby("factor")
    .apply(lambda x: (
        x["estimate"].mean()/ 
          smf.ols("estimate ~ 1", x)
          .fit(cov_type="HAC", cov_kwds={"maxlags": 4}).bse
      )
    )
    .reset_index()
    .rename(columns={"Intercept": "t_stat_NW"})
  )
  # express risk premium in percentage
  price_of_risk['risk_premium'] = price_of_risk['risk_premium'] * 100
  return price_of_risk.merge(price_of_risk_NW, on="factor").round(3)

In [31]:
# reg_panel = pd.read_pickle(
#     PROCESSED_DATA_PATH / "panel_main.pickle.zip", compression="zip"
# )
# reg_panel.columns

In [32]:
# load factors
ff3 = pd.read_csv(PROCESSED_DATA_PATH/"FF3.csv") 
ltw3 = pd.read_csv(PROCESSED_DATA_PATH/"LTW3.csv")

# load the regression panel dataset
reg_panel = pd.read_pickle(
    PROCESSED_DATA_PATH / "panel_main.pickle.zip", compression="zip"
)

reg_panel["daily_supply_return"] = reg_panel["supply_rates"] / 365.2425


def get_weekly_panel(reg_panel):
    ### Get test assets returns 
    reg_panel['Date'] = pd.to_datetime(reg_panel['Date'])
    reg_panel['Year'] = reg_panel['Date'].dt.year
    reg_panel['Week'] = reg_panel['Date'].dt.isocalendar().week

    # add daily simple returns and convenience yield
    reg_panel = calculate_period_return(df_panel=reg_panel, freq=1, simple_dollar_ret=False)

    # Change into weekly returns
    reg_panel = calculate_weekly_returns(df_panel=reg_panel)

    reg_panel['weekly_dollar_ret'] = reg_panel.groupby(['Year', 'Week'])['weekly_dollar_ret'].transform(
            lambda x: x.clip(lower=x.quantile(0.01), upper=x.quantile(0.99))
    )
    return reg_panel
    
# stable non-stable info dict
stable_nonstable_info = {
    "stablecoin": reg_panel[reg_panel["Token"].isin(STABLE_DICT.keys())],
    "non-stablecoin": reg_panel[~reg_panel["Token"].isin(STABLE_DICT.keys())],
    # "all": reg_panel,
}

for panel_info, df_panel in stable_nonstable_info.items():
    for simple_dollar_ret in [True]:
        for dominance in DEPENDENT_VARIABLES[:1]: 
            print(f"Fama McBeth {panel_info} {simple_dollar_ret} {dominance}")
            dominance_factor = build_dominance_factor(
                    reg_panel=df_panel,
                    brk_pt_lst=[0.8],
                    dominance_var=dominance,
                    zero_value_portfolio=False,
                    simple_dollar_ret=simple_dollar_ret,
                )
            dominance_factor.drop(['P1', 'P2'], axis=1, inplace=True)
            dominance_factor.rename(columns={"P2 - P1": "DOM"}, inplace=True)

            assets_panel = get_weekly_panel(df_panel)
            # merge all factors
            data_fama_macbeth = pd.merge(dominance_factor, ff3, on=['Year', 'Week'], how="left")
            data_fama_macbeth = pd.merge(data_fama_macbeth, ltw3, on=['Year', 'Week'], how="left")
            # merge factors with returns
            data_fama_macbeth = pd.merge(data_fama_macbeth, assets_panel, on=['Year', 'Week'], how="left")
            data_fama_macbeth.rename(columns={"weekly_dollar_ret": "dollar_ret", "weekly_ret":"convenience_ret"}, inplace=True)
            data_fama_macbeth = data_fama_macbeth.dropna()

            # run the Fama-Macbeth regression
            data_fama_macbeth['excess_ret'] = data_fama_macbeth['dollar_ret'] - data_fama_macbeth['RF']
            fama_macbeth = reg_fama_macbeth(data_fama_macbeth, formula="excess_ret ~ CMKT + CMOM + CSIZE + DOM")
            display(fama_macbeth)


Fama McBeth stablecoin True volume_ultimate_share


,factor,risk_premium,t_stat,t_stat_NW
0,CMKT,-0.291,-0.827,-0.864
1,CMOM,0.857,0.985,1.010
2,CSIZE,-0.388,-1.038,-1.057
3,DOM,-0.016,-1.284,-1.319
4,Intercept,10.599,0.938,0.963


Fama McBeth non-stablecoin True volume_ultimate_share


,factor,risk_premium,t_stat,t_stat_NW
0,CMKT,0.817,5.709,5.641
1,CMOM,-0.002,-0.026,-0.024
2,CSIZE,0.265,3.215,2.395
3,DOM,-0.062,-0.419,-0.435
4,Intercept,3.740,3.249,2.370


In [ ]:
import statsmodels.api as sm
factors_data = pd.merge(dominance_factor,ltw3, how='left', on=['Year', 'Week'])
factors_data = factors_data.dropna()
y = factors_data['DOM']
X = factors_data[['CMKT', 'CMOM', 'CSIZE']]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    DOM   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.021
Method:                 Least Squares   F-statistic:                   0.07136
Date:                Sat, 01 Mar 2025   Prob (F-statistic):              0.975
Time:                        23:50:36   Log-Likelihood:                -54.251
No. Observations:                 134   AIC:                             116.5
Df Residuals:                     130   BIC:                             128.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0382      0.034     -1.126      0.2

: 